In [1]:
import pandas as pd
from unidecode import unidecode
import re
import numpy as np
pd.set_option("display.max_colwidth", 200)
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

In [2]:
filename = "../DATA/ANALYSIS/all_reviews_all_companies_translated.xlsx"
df = pd.read_excel(filename, index_col=0, engine="openpyxl")
df.head()

,company,review_id,rating_star,date,title,review,customer_location,n_reviews_customer_hide,is_verified,verification_source_hide,review_source_hide,verification_level_hide,reply_content_hide,rating_star_cleaned_hide,n_reviews_customer_cleaned_hide,date_year_month_hide,text_cleaned_hide,page,text_cleaned_hide_EN
0,Lauritz.com,6062edd1f85d7508703d58f0,1 étoile : mauvais,2021-03-30T09:22:25+00:00,Håbløst og ulovligt !,Håbløst !\nSender dem en mail om fortrydelse og får intet svar. Må rykke i en uge. Køber et teak bord som viser sig at være palisander og nogle palisander skabe som stinker af røg!\nFordi jeg samt...,DK,26 avis,False,NaN,NaN,NaN,NaN,1,2,2021-03,håbløst og ulovligt ! håbløst !\nsender dem en mail om fortrydelse og får intet svar. må rykke i en uge. køber et teak bord som viser sig at være palisander og nogle palisander skabe som stinker a...,1,hopelessly and illegal! hopeless !\nSends them an email on regret and get no answer. May jerk for a week. Buyer a teak table that turns out to be rosewood and some rosewood cabinets like stinks of...
1,Lauritz.com,606218d2f85d7508703c5d8c,5 étoiles : excellent,2021-03-29T18:13:38+00:00,er fuldstændig styr på det hele,NaN,DK,18 avis,False,NaN,NaN,NaN,NaN,5,1,2021-03,er fuldstændig styr på det hele,1,is complete track of it all
2,Lauritz.com,60616a7ff85d7508703acce2,3 étoiles : moyen,2021-03-29T05:49:51+00:00,Lauritz .com har et gevaldigt problem…,Lauritz .com har et gevaldigt problem med transportøren. Det er ALT for dyrt og ALT for langsomt!\nHvorfor skal jeg vente en uge på lev. fra Århus til jeres afdeling i Herlev og betale mere end 38...,DK,10 avis,False,NaN,NaN,NaN,NaN,3,1,2021-03,lauritz .com har et gevaldigt problem… lauritz .com har et gevaldigt problem med transportøren. det er alt for dyrt og alt for langsomt!\nhvorfor skal jeg vente en uge på lev. fra århus til jeres ...,1,Lauritz .com has a tremendous problem ... Lauritz .com has a tremendous problem with the carrier. It's too expensive and too slow!\nWhy should I wait a week in Lev. From Aarhus to your department ...
3,Lauritz.com,605cdf6af85d750870350ea5,1 étoile : mauvais,2021-03-25T19:07:22+00:00,Jeg har købt en vare via jeres auktion,"Jeg har købt en vare via jeres auktion, og min oplevelse med jeres kundeservice i den forbindelse lader mildt sagt en hel del tilbage at ønske – ja, jeg vil endda sige at servicen var under al kri...",DK,1 avis,False,NaN,NaN,NaN,NaN,1,1,2021-03,"jeg har købt en vare via jeres auktion jeg har købt en vare via jeres auktion, og min oplevelse med jeres kundeservice i den forbindelse lader mildt sagt en hel del tilbage at ønske – ja, jeg vil ...",1,"I bought an item through your auction I have bought an item through your auction, and my experience with your customer service in this connection lets mildly said a great deal to want - yes, I wou..."
4,Lauritz.com,605bb816f85d750870332126,5 étoiles : excellent,2021-03-24T22:07:18+00:00,Det positive først,"Det positive først. 2 gange indenfor relativ kort tid har jeg solgt 2 genstande på Lauritz.com, Aarhus. Begge gange med hammerslag over vurderingen, og \nvel at mærke begge gange med en værdi over...",DK,1 avis,False,NaN,NaN,NaN,"Kære Per Jensen.\nTusind tak for den gode anmeldelse, og fordi du tager dig tid til at skrive den. Det betyder meget for os.\nDu har ret i, at fragtpriser kan være en barriere for at handle i vore...",5,1,2021-03,"det positive først det positive først. 2 gange indenfor relativ kort tid har jeg solgt 2 genstande på lauritz.com, aarhus. begge gange med hammerslag over vurderingen, og \nvel at mærke begge gang...",1,"It first positively the positive first. 2 times within relatively short time I have sold 2 objects on Lauritz.com, Aarhus. both times with hammer layer over the assessment, and\nTo mark both times..."


In [3]:
def clean_text(text):
    return re.sub("[^a-z\s]", " ", unidecode(text.lower()))

df["text_cleaned_hide_EN"] = df["text_cleaned_hide_EN"].apply(clean_text)
# df["text_cleaned_hide"] = df["review"].apply(clean_text)

In [4]:
def print_words(df, n=200):
    df_words = df["text_cleaned_hide_EN"].str.split(expand=True).stack()
    df_words_supn = df_words[df_words.apply(len) > 3]
    df_freq_words = df_words_supn.value_counts()
    pd.set_option('display.max_rows', n)
    print(df_freq_words.head(n))

In [5]:
print_words(df, n=200)

that             7614
have             6483
with             5122
good             4031
they             3608
there            2606
when             2526
lauritz          2425
tradera          2185
from             2133
easy             2050
service          1968
sell             1910
this             1820
very             1806
just             1799
things           1646
time             1562
been             1550
many             1510
their            1507
only             1433
item             1430
customer         1381
then             1362
sold             1354
money            1353
what             1313
more             1306
about            1301
also             1298
will             1269
should           1266
price            1232
well             1210
after            1180
them             1112
would            1108
your             1008
super             980
which             970
think             964
auction           958
could             952
really            932
works     

In [6]:
dict_keywords = {
    "delivery": ["deliver", "packag", "expedition", "shipping", "transport"],
    "pickup": ["pick"],
    "service": ["customer service", "service", "support", "mail", "call", "respon", "message", "staff",
               "guidance", "treatment", "contact", "profession", "employe", "listen"],
    "products": ["damage", "destroy", "defective", "scratch", "error",
                "good product", "bought good", "picture", "produced", "photo",
                "description", "deficient", "quality", "broke", "break", "dirty"],
    "new_products": ["newly", "produced"],
    "debt_collection": ["collection", "debt"],
    "scam": [" bot ", " bots ", "fraud", "cheat", "deceive", "counterfeit", " copy", "scam", "police",
            " law ", " laws ", "fake"],
    "website": ["website", "search engine", "surf", "layout", "design"],
    "platform": ["ebay"],
    "mobile": [" app ", "mobile"],
    "payment": ["payout", "payoff", "paypal", "payment", "transfer"],
    "price": ['expensive', ' fee', "hidden cost", "reasonable", "commission"],
    "reimbursement": ["money back", "refund"],
    "estimate": ["estimate", "assess", "submit"],
    "account": ["account", "profile", "register", "validat"],
    "review_process": ["judgment", "review"],
    "invoice": ["invoice"],
    "ads": [" ads", " ad ", "adverti"],
    "blue_newspaper": ["blue newspaper"],
    "yellow": ["yellow"],
    "auction": [ "minimum", "hammer", "premium"],
}

dict_keywords["priceAll"] = dict_keywords["price"] + dict_keywords["auction"]
dict_keywords["deliveryAll"] = dict_keywords["delivery"] + dict_keywords["pickup"]
dict_keywords["interface"] = dict_keywords["website"] + dict_keywords["mobile"]
dict_keywords["adsAll"] = dict_keywords["ads"] + dict_keywords["blue_newspaper"]
dict_keywords["productsAll"] = dict_keywords["products"] + dict_keywords["new_products"]


def _aux_get_keyword(pn, list_kw):
    if pd.notna(pn):
        for k in list_kw:
            if k in pn:
                return k
    return ""


def get_cat(row):
    categories = []
    for cat, list_kwds in dict_keywords.items():
        if row[f"keyword_{cat}"] in list_kwds:
            categories.append(cat)
    return "|".join(categories)

for cat, list_kwds in dict_keywords.items():
    df[f"keyword_{cat}"] = df[
        "text_cleaned_hide_EN"].apply(lambda pn: _aux_get_keyword(pn, list_kwds))

    
df["category"] = df.apply(get_cat, axis=1)

In [7]:
df["category"].apply(lambda x: x.split("|")).explode().value_counts()

service            3151
                   2837
adsAll             1833
ads                1746
priceAll           1431
price              1208
deliveryAll        1100
interface          1005
productsAll         953
products            943
scam                900
delivery            883
payment             656
website             621
account             571
estimate            472
review_process      471
mobile              444
auction             346
pickup              332
yellow              298
reimbursement       198
platform            147
blue_newspaper      126
invoice              92
new_products         85
debt_collection      67
Name: category, dtype: int64

In [8]:
print(df[df.category == ""].shape[0])
df[df.category == ""].shape[0] / df.shape[0]

2837


0.31056376573617955

In [9]:
df_w = df[(df.category == "") & (df.rating_star_cleaned_hide < 4)]
print(df_w.shape[0])
df_w.shape[0] / df[(df.rating_star_cleaned_hide < 4)].shape[0]

304


0.09282442748091603

In [10]:
for cat in dict_keywords.keys():
    print(f"Category {cat}: ", (df[f"keyword_{cat}"] != "").sum())

Category delivery:  883
Category pickup:  332
Category service:  3151
Category products:  943
Category new_products:  85
Category debt_collection:  67
Category scam:  900
Category website:  621
Category platform:  147
Category mobile:  444
Category payment:  656
Category price:  1208
Category reimbursement:  198
Category estimate:  472
Category account:  571
Category review_process:  471
Category invoice:  92
Category ads:  1746
Category blue_newspaper:  126
Category yellow:  298
Category auction:  346
Category priceAll:  1431
Category deliveryAll:  1100
Category interface:  1005
Category adsAll:  1833
Category productsAll:  953


In [11]:
print_words(df[(df.category == "") & (df.rating_star_cleaned_hide <4)], n=200)

that             126
have             118
with              87
good              65
they              62
there             57
when              49
block             45
more              44
tradera           40
time              40
what              39
just              38
many              37
very              37
money             36
think             33
search            33
side              33
this              33
should            32
sold              31
from              30
only              30
page              29
will              28
about             28
item              27
little            27
want              26
lauritz           25
things            25
their             25
before            25
been              24
must              24
could             23
them              23
sell              22
better            22
other             22
find              22
much              21
people            21
something         21
then              20
price             20
become       

In [12]:
print("\n\n\n".join(list(df[
    df.apply(
        lambda x: ("people" in x["text_cleaned_hide_EN"].lower()),# and (x["category"] == ""),
        axis=1
    )]["text_cleaned_hide_EN"])))

i got my watch rated at     kr and     i got my watch rated at     kr and agreed that i would have     kr as absolute minimum in your pocket 
i wrote under the contract in which the agreed minimum price was     kr  which i went from was it as i would get in his pocket after everything paid off my oral agreement with the employee  it should turn out that it was not so headed and they took both hammergsgesby plus     of the amount 
of course  i immediately contacted their customer service as completely rejected to meet my requirement to compensate the missing amount so that the agreement was complied with 
the customer service at lauritz com is with it in my optics completely indifferent to their customers and you really get a good insight into how bad a business it is when the problems arise with them  their rudeness  indifference and lacking ability to solve problems when they make mistakes against the people who sell with them show quality of their business  very very poor quality and

In [13]:
print("\n\n\n".join(list(df[
    df.apply(
        lambda x: (x["category"] == "") and (x["rating_star_cleaned_hide"] < 4),
        axis=1
    )]["text_cleaned_hide_EN"])))

do not recommend really practice experience  first and last time i approach you 


they are about to go bankrupt         they are about to go bankrupt          good advice hurry you away 


herlev department has very difficulties     herlev department has very difficulty finding out when to c   d on a product if in doubt comes there c   d at  it is their own removal  


the extras are worthless  you better stay away from them  i announced three things on tradera out of which two were brand new and i spent around     sek for extras like highlights etc  
i got no bids  no questions  no nothing 
it felt like i had been the only person with his stuff on the platform 
even though the concept is good  you better stay away from the extras they are not worth a penny 


completely useless to wait     weeks on     completely useless to wait     weeks to get money from a sales firmly the buyer has already got his item just because they will serve interest  it is not the first time it happens eith

In [14]:
df.to_excel("../DATA/ANALYSIS/topics_analysis_all V2.xlsx")